In [ ]:
import sys
import os

# Add the path to the directory where your module is located
module_path = os.path.abspath(os.path.join('..', 'C:\\Users\\pier1\\OneDrive\\Desktop\\uni\\Master\\2.Semester\\Machine Learning (WIWI)\\Project\\Data for depression\\'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import your module
from helpers import *

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from NN import *

In [ ]:
df = pd.read_csv('..\data\Threshold_3_Operator_-_Depressionfeature_BP_PHQ_9_PercentofDataset_100.csv')
print_information(df)
# 0.9 train, 0.1 test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# 0.8 train, 0.2 validation
#train_df, validation_df = train_test_split(train_df, train_size=0.8, random_state=42)

train_features, train_targets = df_to_tensor(train_df,features_column='FEATURES', target_col='Depression')
#validation_features, validation_targets = df_to_tensor(validation_df, features_column='FEATURES', target_col='Depression')
test_features, test_targets = df_to_tensor(test_df, features_column='FEATURES', target_col='Depression')

#print(train_features.shape, train_targets.shape, validation_features.shape, validation_targets.shape, test_features.shape, test_targets.shape)
print(train_features.shape, train_targets.shape, test_features.shape, test_targets.shape)

train_dataset = CustomDataset(train_features, train_targets)
#validation_dataset = CustomDataset(validation_features, validation_targets)
test_dataset = CustomDataset(test_features, test_targets)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Configuration
input_size = 56
hidden_size = 128
num_epochs = 60
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Model is bad at 94 % accuracy, because 94 percent of the data is not depressed (PHQ_9)
# Initialize the model, criterion, and optimizer
model = Depression_Classifier_v_2(input_size, hidden_size).to(device)
#criterion = nn.BCEWithLogitsLoss()  # Combines sigmoid and binary cross-entropy loss
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizers = ['Adam', 'SGD']
train_model(model, train_dataloader, criterion, optimizer, num_epochs, device)
evaluate_model(model, test_dataloader, criterion, device)

#results = {}
#results = cross_validation(train_dataloader, validation_dataloader, input_size, hidden_size, criterion, optimizers, device, results)

           

In [ ]:
df = pd.DataFrame()
models_df = []
optimizer_df = []
learning_rate_df = []
momentum_df = []
accuracy_df = []
validation_loss_df = []
for key,value in results.items():
    accuracy_df.append(value['accuracy'])
    validation_loss_df.append(value['validation_loss'])
    opti = key[1]
    models_df.append(key[0])
    optimizer_df.append(opti)
    learning_rate_df.append(key[2])
    if opti == 'SGD':
        momentum_df.append(key[3])
    else:
        momentum_df.append(None)
        
df['Model'] = models_df
df['Optimizer'] = optimizer_df
df['Learning Rate'] = learning_rate_df
df['Momentum'] = momentum_df
df['Validation Loss'] = validation_loss_df
df['Accuracy'] = accuracy_df


In [ ]:
print(df)

In [ ]:
df.to_csv('FNN_MH_PHQ_S_minus.csv', index=False)

Use the csv data to compare the different models and cross-validations

In [ ]:
bp_phq_9 = pd.read_csv('FNN_BP_PHQ_9_minus.csv')
mh_phq_s = pd.read_csv('FNN_MH_PHQ_S_minus.csv')

max_index_phq9 = bp_phq_9['Accuracy'].idxmax()
max_index_mh_phq_s = mh_phq_s['Accuracy'].idxmax()

print(bp_phq_9.loc[max_index_phq9])
print(mh_phq_s.loc[max_index_mh_phq_s])

- Based on the cross-validation with different learning rates, momentum values, optimizers, models and 30 epochs the best accuracy is based of
- Model_2, Adam, lr = 0.001 with a accuracy of 94% with the depression_classifier PHQ_9
- Model_2, Adam, lr = 0.001 with a accuracy of 83% with the depression_classifier MH_PHQ_s
- Using these values we run for 60 epochs
- Model_2, Adam, lr = 0.001 with a accuracy of 89% with the depression_classifier MH_PHQ_s
- Model_2, Adam, lr = 0.001 with a accuracy of 95% with the depression_classifier PHQ_9

In [ ]:
torch.save(model.state_dict(), 'BP_PHQ_9.pth')